In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Loading the Data Into Memory (with Keras)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
from numpy import mean, std 
from tensorflow.keras.models import load_model,Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow

In [ ]:
num_classes=24


In [2]:
train_path ='asl_data/sign_mnist_train.csv'
valid_path ='asl_data/sign_mnist_valid.csv'

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks (2)/Quynh Hoa

/content/drive/MyDrive/Colab Notebooks (2)/Quynh Hoa


In [4]:
train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

NameError: ignored

In [ ]:
def load_dataset(train,valid):
	train_df = pd.read_csv(train)
	valid_df = pd.read_csv(valid)

	y_train = train_df['label']
	y_valid = valid_df['label']
	del train_df['label']
	del valid_df['label']

	x_train = train_df.values
	x_valid = valid_df.values
	
	x_train = x_train.reshape(-1,28,28,1)
	x_valid = x_valid.reshape(-1,28,28,1)

	return (x_train,y_train),(x_valid,y_valid)
	
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

def define_model():
	model = Sequential()
	model.add(Dense(units = 512, activation='relu',input_shape=(28,28,1)))
	model.add(Dense(units = 512, activation='relu'))
	model.add(Flatten())
	model.add(Dense(units=num_classes, activation = 'softmax'))
	
	model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
	model.summary()
	return model 
	
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	count = 0
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model()
		# select rows for train and test
		x_train, y_train, x_valid, y_valid = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		
		# fit model
		history = model.fit(x_train, y_train, epochs=10, validation_data=(x_valid, y_valid))
		# evaluate model
		_, acc = model.evaluate(x_valid, y_valid, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		scores.append(acc)
		histories.append(history)
		model.save('sign_mnist'+'_'+'version'+'_'+str(count)+'.h5')
		count += 1
	return scores, histories
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		plt.subplot(2, 1, 1)
		plt.title('Cross Entropy Loss')
		plt.plot(histories[i].history['loss'], color='blue', label='train')
		plt.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		plt.subplot(2, 1, 2)
		plt.title('Classification Accuracy')
		plt.plot(histories[i].history['accuracy'], color='blue', label='train')
		plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	plt.show()

def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	plt.boxplot(scores)
	plt.show()

def evaluate(data_path):
	# load dataset
	(x_train, y_train), (x_valid,y_valid) = load_dataset(data_path)
	# prepare pixel data
	x_train, x_valid = prep_pixels(x_train, x_valid)
	y_train = keras.utils.to_categorical(y_train,num_classes)
	y_valid = keras.utils.to_categorical(y_valid,num_classes)

	# evaluate model
	scores, histories = evaluate_model(x_train, y_train)
	# learning curves
	summarize_diagnostics(histories)
	# summarize estimated performance
	summarize_performance(scores)

def train_model(train_path,valid_path):
	# load dataset
	(x_train, y_train), (x_valid, y_valid) = load_dataset(train_path,valid_path)

	y_train = keras.utils.to_categorical(y_train,num_classes)
	y_valid = keras.utils.to_categorical(y_valid,num_classes)

	# prepare pixel data
	x_train, x_valid = prep_pixels(x_train, x_valid)
	# define model
	model = define_model()
	# fit model
	model.fit(x_train, y_train, epochs=20, validation_data=(x_valid,y_valid))

	model.save('sign_mnist.h5')


In [ ]:
#Show all classes
s = set()
(x_train,y_train),(x_valid,y_valid) = load_dataset(train_path, valid_path)
for idx in range(27455):
    label = y_train[idx]
    prev = len(s)
    s.add(label)
    cont = len(s)
    if (prev is not cont):
        plt.subplot(1, num_classes+1, len(s)+1)
        plt.title(label, fontdict={'fontsize':10})
        plt.axis('off')
        plt.imshow(x_train[idx], cmap='gray')

In [ ]:
train_model(train_path,valid_path)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 28, 28, 512)       1024      
                                                                 
 dense_10 (Dense)            (None, 28, 28, 512)       262656    
                                                                 
 flatten_3 (Flatten)         (None, 401408)            0         
                                                                 
 dense_11 (Dense)            (None, 24)                9633816   
                                                                 
Total params: 9,897,496
Trainable params: 9,897,496
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
858/858 [==============================] - 23s 26ms/step - loss: 0.7546 - accuracy: 0.7897 - val_loss: 1.5976 - val_accuracy: 0.6758
Epoch 2/20
858/858 [============

In [ ]:
!ls

asl_data    FaceRecognition	       LSTM_Basic	 sign_mnist.h5
asl_images  GenerateMusic_Transformer  Pytorch_Tutorial  sign_mnist.ipynb


In [6]:
import cv2 
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import load_model
import numpy as np

model = load_model('sign_mnist.h5')
def load_image(filename):
	# load the image
	img = load_img(filename, grayscale=True, target_size=(28, 28))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(1, 28, 28, 1)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img
img = load_image('asl_images/b.png')
predict_value = model.predict(img)
digit = np.argmax(predict_value)
print(digit)

1/1 [==============================] - 0s 129ms/step
1


In [ ]:
len(predict_value[0])

In [ ]:
for x in range(len(predict_value)):
    